# Tutorial 16: Stress Relaxation with FluiditySaramitoLocal

## Learning Objectives

This notebook demonstrates stress relaxation analysis using the FluiditySaramitoLocal model with tensorial elastoviscoplastic dynamics:

1. **Non-Exponential Decay**: Understand how thixotropic aging creates stretched exponential relaxation
2. **Tensorial Relaxation**: Track full stress tensor [τ_xx, τ_yy, τ_xy] decay from imposed strain
3. **Fluidity Evolution**: Observe structural recovery (f → f_eq) during quiescent aging
4. **Elastic Jump**: Measure initial elastic response from Maxwell backbone
5. **Normal Stress Decay**: Analyze how N₁ relaxes alongside shear stress
6. **NLSQ + Bayesian**: Calibrate relaxation time λ and aging parameters from transient data
7. **Model Comparison**: Contrast minimal vs full coupling modes for yield stress aging

**Key Physics**: After a step strain γ₀ is imposed at t=0, the stress relaxes via σ(t) = G(t)γ₀ where the relaxation modulus G(t) decays non-exponentially due to time-dependent fluidity f(t). This signature distinguishes thixotropic materials from simple viscoelastic fluids.

**Saramito-Specific Features**:
- Tensorial Upper Convected Maxwell (UCM) backbone for viscoelasticity
- Von Mises yield criterion: α = max(0, 1 - τ_y/|τ|)
- Aging-dependent yield stress: τ_y(f) in full coupling mode
- Normal stress components: N₁ = τ_xx - τ_yy from UCM

## Google Colab Setup

Run this cell if using Google Colab to install RheoJAX:

In [ ]:
# Uncomment and run in Google Colab
# !pip install rheojax jaxopt optax arviz

## Setup and Imports

In [ ]:
# JAX float64 configuration (CRITICAL: must come before any JAX imports)
from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()

# Standard imports
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# RheoJAX imports
from rheojax.models.fluidity import FluiditySaramitoLocal
from rheojax.core.data import RheoData
from rheojax.logging import configure_logging, get_logger

# Bayesian inference
import arviz as az

# Add utils to path for tutorial helpers
IN_COLAB = 'google.colab' in sys.modules
if not IN_COLAB:
    utils_path = Path("..") / "utils"
    if utils_path not in [Path(p) for p in sys.path]:
        sys.path.insert(0, str(utils_path))

from fluidity_tutorial_utils import (
    get_output_dir,
    save_fluidity_results,
    print_convergence_summary,
    print_parameter_comparison,
    compute_fit_quality,
)

# Configure logging
configure_logging(level="INFO")
logger = get_logger(__name__)

# Plot styling
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Set random seeds
np.random.seed(42)
key = jax.random.PRNGKey(42)

print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")
print(f"Float64 enabled: {jax.config.jax_enable_x64}")

## Theory: Saramito Tensorial Stress Relaxation

### Governing Equations

After a step strain γ₀ is imposed at t=0 with no further deformation (γ̇=0), the stress evolves via:

1. **Upper Convected Maxwell (UCM) Relaxation**:
   $$\boldsymbol{\tau} + \lambda(f) \frac{D\boldsymbol{\tau}}{Dt} = 0 \quad \text{(no flow)}$$
   
   Simplifies to:
   $$\frac{d\boldsymbol{\tau}}{dt} = -\frac{1}{\lambda(f)} \boldsymbol{\tau}$$

2. **Fluidity-Dependent Relaxation Time**:
   $$\lambda(f) = \frac{\eta_0}{G} \cdot \frac{1}{f}$$
   
   where η₀ is zero-shear viscosity, G is elastic modulus, and f is fluidity.

3. **Aging Evolution (Quiescent, γ̇=0)**:
   $$\frac{df}{dt} = -\frac{f - 1}{t_{eq}}$$
   
   Structure recovers: f(t) → 1 (fully structured state).

4. **Von Mises Check** (typically inactive during relaxation):
   $$\alpha = \max\left(0, 1 - \frac{\tau_y(f)}{|\boldsymbol{\tau}|}\right)$$
   
   For most relaxation scenarios, |τ| < τ_y quickly → α ≈ 0 → viscoelastic relaxation only.

### Key Observables

1. **Relaxation Modulus**: G(t) = σ(t)/γ₀ decays non-exponentially
2. **Stretched Exponential**: Faster decay at early times (high f) → slower decay (low f)
3. **Normal Stress Decay**: N₁(t) = τ_xx(t) - τ_yy(t) follows similar dynamics
4. **Fluidity Recovery**: f(t) exponentially approaches f=1 with timescale t_eq

### Contrast with Maxwell Model

- **Maxwell**: G(t) = G₀ exp(-t/λ) — single exponential with constant λ
- **Saramito**: G(t) ∝ exp(-∫dt/λ(f(t))) — time-varying λ creates curvature on semi-log plot

### Coupling Modes

- **Minimal**: Only λ(f) = λ₀/f varies. Yield stress τ_y constant.
- **Full**: Both λ(f) and τ_y(f) vary. Aging increases yield stress.

## Load Calibrated Parameters

Attempt to load parameters from startup calibration (Tutorial 14). If unavailable, use sensible defaults.

In [ ]:
# Try to load from startup calibration
if IN_COLAB:
    output_dir = Path('/content/outputs/fluidity/saramito_local/relaxation')
    param_file = None  # No pre-calibration in Colab
else:
    output_dir = get_output_dir('saramito_local', 'relaxation')
    param_file = output_dir.parent / 'startup' / 'nlsq_params_startup.json'

output_dir.mkdir(parents=True, exist_ok=True)

if param_file and param_file.exists():
    logger.info(f"Loading calibrated parameters from {param_file}")
    import json
    with open(param_file) as f:
        params = json.load(f)
    
    eta_0 = params.get('eta_0', 100.0)
    tau_y = params.get('tau_y', 50.0)
    lambda_0 = params.get('lambda_0', 1.0)
    t_eq = params.get('t_eq', 10.0)
    b = params.get('b', 0.5)
    n = params.get('n', 1.0)
    m = params.get('m', 0.0)
    
    logger.info(f"Loaded: η₀={eta_0:.2f}, τ_y={tau_y:.2f}, λ₀={lambda_0:.2f}, t_eq={t_eq:.2f}")
else:
    logger.info("No calibrated parameters found, using defaults")
    # Default parameters for demonstration
    eta_0 = 100.0      # Zero-shear viscosity (Pa·s)
    tau_y = 50.0       # Yield stress (Pa)
    lambda_0 = 1.0     # Relaxation time at f=1 (s)
    t_eq = 10.0        # Equilibrium aging time (s)
    b = 0.5            # Rejuvenation coefficient
    n = 1.0            # Shear-rate exponent
    m = 0.0            # Fluidity exponent

print("\n=== Model Parameters ===")
print(f"η₀ (zero-shear viscosity): {eta_0:.2f} Pa·s")
print(f"τ_y (yield stress): {tau_y:.2f} Pa")
print(f"λ₀ (relaxation time): {lambda_0:.2f} s")
print(f"t_eq (aging time): {t_eq:.2f} s")
print(f"b (rejuvenation): {b:.2f}")
print(f"n (shear exponent): {n:.2f}")
print(f"m (fluidity exponent): {m:.2f}")

## Generate Synthetic Relaxation Data

Simulate stress relaxation after a step strain γ₀, showing non-exponential decay from evolving fluidity.

In [ ]:
# Create model with known parameters for data generation
model_true = FluiditySaramitoLocal(coupling="minimal")

# Set true parameters
model_true.parameters['eta_0'].value = eta_0
model_true.parameters['tau_y'].value = tau_y
model_true.parameters['lambda_0'].value = lambda_0
model_true.parameters['t_eq'].value = t_eq
model_true.parameters['b'].value = b
model_true.parameters['n'].value = n
model_true.parameters['m'].value = m

# Relaxation simulation parameters
gamma_0 = 0.1            # Step strain (dimensionless)
t_end = 100.0            # Simulation time (s)
n_points = 200           # Time points

# Generate logarithmic time array (better resolution for decay)
t_relax = np.logspace(-2, np.log10(t_end), n_points)

# Simulate relaxation
logger.info(f"Simulating relaxation after γ₀ = {gamma_0:.3f} step strain")
sigma_0 = 500.0  # Initial stress (Pa) — will be rescaled by model

# Use simulate_relaxation method (returns stress, fluidity)
try:
    stress_true, fluidity_true = model_true.simulate_relaxation(t_relax, gamma_0=gamma_0)
    # Handle potential shape issues
    if stress_true.ndim > 1:
        # Extract shear stress component if tensorial
        tau_xy_true = np.array(stress_true[:, 2]) if stress_true.shape[1] >= 3 else np.array(stress_true.flatten())
        tau_xx_true = np.array(stress_true[:, 0]) if stress_true.shape[1] >= 3 else np.zeros_like(tau_xy_true)
        tau_yy_true = np.array(stress_true[:, 1]) if stress_true.shape[1] >= 3 else np.zeros_like(tau_xy_true)
    else:
        tau_xy_true = np.array(stress_true.flatten())
        tau_xx_true = np.zeros_like(tau_xy_true)
        tau_yy_true = np.zeros_like(tau_xy_true)
    fluidity_true = np.array(fluidity_true)
except AttributeError:
    # Fallback: use predict with test_mode='relaxation'
    logger.warning("simulate_relaxation not available, using predict fallback")
    model_true._test_mode = 'relaxation'
    model_true._gamma_0 = gamma_0
    stress_pred = model_true.predict(t_relax)
    if stress_pred.ndim > 1:
        tau_xy_true = np.array(stress_pred[:, 2]) if stress_pred.shape[1] >= 3 else np.array(stress_pred.flatten())
        tau_xx_true = np.array(stress_pred[:, 0]) if stress_pred.shape[1] >= 3 else np.zeros_like(tau_xy_true)
        tau_yy_true = np.array(stress_pred[:, 1]) if stress_pred.shape[1] >= 3 else np.zeros_like(tau_xy_true)
    else:
        tau_xy_true = np.array(stress_pred.flatten())
        tau_xx_true = np.zeros_like(tau_xy_true)
        tau_yy_true = np.zeros_like(tau_xy_true)
    # Estimate fluidity evolution (exponential aging)
    f_0 = 2.0  # Start with high fluidity (post-flow)
    fluidity_true = 1.0 + (f_0 - 1.0) * np.exp(-t_relax / t_eq)

# Normalize to target initial stress
if tau_xy_true[0] > 0:
    tau_xy_true = tau_xy_true * (sigma_0 / tau_xy_true[0])
    tau_xx_true = tau_xx_true * (sigma_0 / (tau_xy_true[0] + 1e-10))
    tau_yy_true = tau_yy_true * (sigma_0 / (tau_xy_true[0] + 1e-10))

# Add realistic noise (5% relative error)
noise_level = 0.05
noise = noise_level * np.abs(tau_xy_true) * np.random.randn(len(tau_xy_true))
tau_xy_noisy = tau_xy_true + noise

# Compute normal stress difference
N1_true = tau_xx_true - tau_yy_true

# Compute relaxation modulus G(t) = σ(t)/γ₀
G_t_true = tau_xy_true / gamma_0

print(f"\n=== Data Characteristics ===")
print(f"Initial stress: {tau_xy_true[0]:.2f} Pa")
print(f"Final stress: {tau_xy_true[-1]:.2f} Pa")
print(f"Decay ratio: {tau_xy_true[-1]/tau_xy_true[0]:.4f}")
print(f"Initial G(t): {G_t_true[0]:.2f} Pa")
print(f"Final fluidity: {fluidity_true[-1]:.3f}")

In [ ]:
# Visualize relaxation data
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Panel 1: Shear stress relaxation
axes[0, 0].loglog(t_relax, tau_xy_true, 'b-', linewidth=2, label='True')
axes[0, 0].loglog(t_relax, tau_xy_noisy, 'r.', markersize=4, alpha=0.6, label='Noisy (5%)')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Shear Stress τ_xy (Pa)')
axes[0, 0].set_title('Stress Relaxation (Non-Exponential Decay)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, which='both')

# Panel 2: Relaxation modulus G(t)
axes[0, 1].loglog(t_relax, G_t_true, 'g-', linewidth=2)
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Relaxation Modulus G(t) (Pa)')
axes[0, 1].set_title('G(t) = σ(t)/γ₀ (Stretched Exponential)')
axes[0, 1].grid(True, alpha=0.3, which='both')

# Panel 3: Fluidity evolution
axes[1, 0].semilogx(t_relax, fluidity_true, 'purple', linewidth=2)
axes[1, 0].axhline(y=1.0, color='k', linestyle='--', alpha=0.3, label='Equilibrium (f=1)')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Fluidity f (dimensionless)')
axes[1, 0].set_title('Structural Recovery (Aging: f → 1)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Panel 4: Semi-log to show deviation from exponential
axes[1, 1].semilogy(t_relax, tau_xy_true, 'b-', linewidth=2, label='Saramito (non-exp)')
# Reference Maxwell exponential
tau_maxwell = tau_xy_true[0] * np.exp(-t_relax / lambda_0)
axes[1, 1].semilogy(t_relax, tau_maxwell, 'k--', linewidth=1.5, alpha=0.6, 
                    label=f'Maxwell (λ={lambda_0:.1f} s)')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Shear Stress τ_xy (Pa)')
axes[1, 1].set_title('Curvature = Thixotropic Signature')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'synthetic_relaxation_overview.png', dpi=300, bbox_inches='tight')
plt.show()

logger.info(f"Saved overview plot to {output_dir / 'synthetic_relaxation_overview.png'}")

## NLSQ Fitting: Parameter Estimation from Relaxation Data

Fit the model to synthetic relaxation data using NLSQ optimization.

In [ ]:
# Create fresh model for fitting
model = FluiditySaramitoLocal(coupling="minimal")

# Prepare data
rheo_data = RheoData(
    x=t_relax,
    y=tau_xy_noisy,
    test_mode='relaxation'
)

# Set initial guesses (slightly perturbed from truth)
model.parameters['eta_0'].value = eta_0 * 0.8
model.parameters['tau_y'].value = tau_y * 1.2
model.parameters['lambda_0'].value = lambda_0 * 0.9
model.parameters['t_eq'].value = t_eq * 1.1
model.parameters['b'].value = b * 0.95
model.parameters['n'].value = n
model.parameters['m'].value = m

print("Initial guess:")
print(model.parameters)

In [ ]:
# Fit with NLSQ
logger.info("Starting NLSQ optimization for relaxation data...")

result = model.fit(
    rheo_data,
    gamma_0=gamma_0,  # Pass step strain for relaxation simulation
    max_iter=5000,
    ftol=1e-8,
    xtol=1e-8
)

print("\n" + "="*60)
print("NLSQ Fitting Results")
print("="*60)
print(f"Converged: {result.success}")
print(f"Iterations: {result.nit}")
print(f"Final cost: {result.cost:.6e}")
print(f"R²: {result.r_squared:.6f}")
print(f"RMSE: {result.rmse:.6e}")

print("\n=== Fitted Parameters ===")
param_names = ['eta_0', 'tau_y', 'lambda_0', 't_eq', 'b', 'n', 'm']
for name in param_names:
    try:
        fitted_val = model.parameters[name].value
        true_val = model_true.parameters[name].value
        error = 100 * abs(fitted_val - true_val) / true_val if true_val != 0 else 0
        print(f"{name:12s}: {fitted_val:10.4f}  (true: {true_val:10.4f}, error: {error:5.2f}%)")
    except KeyError:
        pass

In [ ]:
# Visualize fit quality
stress_fit = model.predict(t_relax, test_mode='relaxation', gamma_0=gamma_0)

# Handle tensorial output
if stress_fit.ndim > 1:
    tau_xy_fit = np.array(stress_fit[:, 2]) if stress_fit.shape[1] >= 3 else np.array(stress_fit.flatten())
else:
    tau_xy_fit = np.array(stress_fit.flatten())

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Panel 1: Fit on log-log
axes[0].loglog(t_relax, tau_xy_noisy, 'ko', markersize=4, alpha=0.5, label='Data (noisy)')
axes[0].loglog(t_relax, tau_xy_true, 'b--', linewidth=2, label='True', alpha=0.7)
axes[0].loglog(t_relax, tau_xy_fit, 'r-', linewidth=2, label='NLSQ Fit')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Shear Stress τ_xy (Pa)')
axes[0].set_title(f'NLSQ Fit Quality (R² = {result.r_squared:.4f})')
axes[0].legend()
axes[0].grid(True, alpha=0.3, which='both')

# Panel 2: Residuals
residuals = tau_xy_noisy - tau_xy_fit
axes[1].semilogx(t_relax, residuals, 'ro', markersize=3, alpha=0.6)
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1].fill_between(t_relax, -2*result.rmse, 2*result.rmse,
                      alpha=0.2, color='red', label='±2 RMSE')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Residuals (Pa)')
axes[1].set_title('Fit Residuals')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'nlsq_fit.png', dpi=300, bbox_inches='tight')
plt.show()

logger.info(f"Saved NLSQ fit plot to {output_dir / 'nlsq_fit.png'}")

## Bayesian Inference: Parameter Uncertainty Quantification

Use NUTS sampling to quantify parameter uncertainties, using NLSQ fit as warm-start.

In [ ]:
# Bayesian inference with NUTS
logger.info("Starting Bayesian inference with NUTS...")

# Run MCMC (using NLSQ fit as warm-start)
bayes_result = model.fit_bayesian(
    rheo_data,
    gamma_0=gamma_0,
    num_warmup=1000,
    num_samples=2000,
    num_chains=4,
    seed=42
)

logger.info("Bayesian inference complete")

In [ ]:
# Extract posterior summary
from rheojax.utils.bayesian import compute_rhat, compute_ess

posterior = bayes_result.posterior_samples

print("\n" + "="*60)
print("Bayesian Posterior Summary")
print("="*60)

for param_name in param_names:
    if param_name not in posterior:
        continue
    
    samples = posterior[param_name]
    mean = float(jnp.mean(samples))
    std = float(jnp.std(samples))
    q025 = float(jnp.percentile(samples, 2.5))
    q975 = float(jnp.percentile(samples, 97.5))
    
    # Reshape for R-hat/ESS computation (num_chains=4)
    samples_reshaped = samples.reshape(4, -1)
    rhat = float(compute_rhat(samples_reshaped))
    ess = float(compute_ess(samples_reshaped))
    
    true_val = model_true.parameters[param_name].value
    
    print(f"\n{param_name}:")
    print(f"  Mean ± Std:     {mean:.4f} ± {std:.4f}")
    print(f"  95% CI:         [{q025:.4f}, {q975:.4f}]")
    print(f"  True value:     {true_val:.4f}")
    print(f"  R-hat:          {rhat:.4f}")
    print(f"  ESS:            {ess:.0f}")

## ArviZ Diagnostics

Comprehensive MCMC convergence and correlation analysis.

In [ ]:
# Convert to ArviZ InferenceData
idata = az.from_dict(
    posterior={k: v.reshape(4, -1) for k, v in posterior.items() if k in param_names},
    observed_data={"y": tau_xy_noisy}
)

# Summary statistics
print("\nArviZ Summary:")
summary = az.summary(idata, hdi_prob=0.95)
print(summary)

# Check convergence
rhat_max = summary['r_hat'].max()
ess_min = summary['ess_bulk'].min()

print(f"\nMax R-hat: {rhat_max:.4f} (should be < 1.01)")
print(f"Min ESS: {ess_min:.0f} (should be > 400)")

if rhat_max < 1.01 and ess_min > 400:
    print("✓ Convergence achieved!")
else:
    print("⚠ Convergence issues detected. Consider increasing num_warmup/num_samples.")

In [ ]:
# Trace plots (chain mixing)
key_params = ['eta_0', 'tau_y', 'lambda_0', 't_eq', 'b']
available_params = [p for p in key_params if p in posterior]

az.plot_trace(
    idata,
    var_names=available_params,
    compact=True,
    figsize=(14, 10)
)
plt.tight_layout()
plt.suptitle('MCMC Trace Plots (Chain Mixing)', y=1.00, fontsize=14)
plt.savefig(output_dir / 'trace_plot.png', dpi=300, bbox_inches='tight')
plt.show()

logger.info(f"Saved trace plot to {output_dir / 'trace_plot.png'}")

In [ ]:
# Pair plot (parameter correlations)
corr_params = ['lambda_0', 't_eq', 'eta_0']
available_corr = [p for p in corr_params if p in posterior]

if len(available_corr) >= 2:
    az.plot_pair(
        idata,
        var_names=available_corr,
        kind='hexbin',
        marginals=True,
        figsize=(10, 10)
    )
    plt.suptitle('Parameter Correlations (Relaxation Data)', y=1.00, fontsize=14)
    plt.tight_layout()
    plt.savefig(output_dir / 'pair_plot.png', dpi=300, bbox_inches='tight')
    plt.show()
    logger.info(f"Saved pair plot to {output_dir / 'pair_plot.png'}")

In [ ]:
# Forest plot (credible intervals)
az.plot_forest(
    idata,
    var_names=available_params,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 6)
)
plt.title('95% Credible Intervals (Forest Plot)')
plt.tight_layout()
plt.savefig(output_dir / 'forest_plot.png', dpi=300, bbox_inches='tight')
plt.show()

logger.info(f"Saved forest plot to {output_dir / 'forest_plot.png'}")

## Tensorial Stress Decomposition

Analyze the relaxation of all stress components and normal stress differences.

In [ ]:
# Predict with posterior mean parameters
model_post = FluiditySaramitoLocal(coupling="minimal")
for name in param_names:
    if name in posterior:
        model_post.parameters[name].value = float(jnp.mean(posterior[name]))

stress_post = model_post.predict(t_relax, test_mode='relaxation', gamma_0=gamma_0)

# Extract components
if stress_post.ndim > 1:
    tau_xx_post = np.array(stress_post[:, 0]) if stress_post.shape[1] >= 3 else np.zeros(len(t_relax))
    tau_yy_post = np.array(stress_post[:, 1]) if stress_post.shape[1] >= 3 else np.zeros(len(t_relax))
    tau_xy_post = np.array(stress_post[:, 2]) if stress_post.shape[1] >= 3 else np.array(stress_post.flatten())
else:
    tau_xy_post = np.array(stress_post.flatten())
    tau_xx_post = np.zeros_like(tau_xy_post)
    tau_yy_post = np.zeros_like(tau_xy_post)

N1_post = tau_xx_post - tau_yy_post

# Plot tensorial evolution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# τ_xy (shear stress)
axes[0, 0].loglog(t_relax, tau_xy_true, 'b--', linewidth=2, label='True', alpha=0.7)
axes[0, 0].loglog(t_relax, tau_xy_post, 'r-', linewidth=2, label='Posterior Mean')
axes[0, 0].loglog(t_relax, tau_xy_noisy, 'ko', markersize=3, alpha=0.3, label='Data')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('τ_xy (Pa)')
axes[0, 0].set_title('Shear Stress Relaxation')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, which='both')

# Normal stress components
axes[0, 1].loglog(t_relax, np.abs(tau_xx_post) + 1e-10, 'b-', linewidth=2, label='τ_xx')
axes[0, 1].loglog(t_relax, np.abs(tau_yy_post) + 1e-10, 'r-', linewidth=2, label='τ_yy')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Normal Stress Components (Pa)')
axes[0, 1].set_title('τ_xx and τ_yy Relaxation')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, which='both')

# N₁ relaxation
axes[1, 0].semilogx(t_relax, N1_post, 'g-', linewidth=2, label='N₁ = τ_xx - τ_yy')
axes[1, 0].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('N₁ (Pa)')
axes[1, 0].set_title('First Normal Stress Difference Relaxation')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Relaxation modulus G(t)
G_t_post = tau_xy_post / gamma_0
axes[1, 1].loglog(t_relax, G_t_post, 'purple', linewidth=2, label='G(t) = σ(t)/γ₀')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Relaxation Modulus G(t) (Pa)')
axes[1, 1].set_title('Relaxation Modulus (Posterior Mean)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig(output_dir / 'tensorial_relaxation.png', dpi=300, bbox_inches='tight')
plt.show()

logger.info(f"Saved tensorial analysis to {output_dir / 'tensorial_relaxation.png'}")

## Coupling Mode Comparison: Minimal vs Full

Compare relaxation behavior with and without aging-dependent yield stress.

In [ ]:
# Create model with full coupling
model_full = FluiditySaramitoLocal(coupling="full")
for name in param_names:
    if name in posterior:
        model_full.parameters[name].value = float(jnp.mean(posterior[name]))

stress_full = model_full.predict(t_relax, test_mode='relaxation', gamma_0=gamma_0)

# Extract shear stress
if stress_full.ndim > 1:
    tau_xy_full = np.array(stress_full[:, 2]) if stress_full.shape[1] >= 3 else np.array(stress_full.flatten())
else:
    tau_xy_full = np.array(stress_full.flatten())

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Log-log comparison
axes[0].loglog(t_relax, tau_xy_post, 'b-', linewidth=2, label='Minimal (λ only)')
axes[0].loglog(t_relax, tau_xy_full, 'r--', linewidth=2, label='Full (λ + τ_y aging)')
axes[0].loglog(t_relax, tau_xy_noisy, 'ko', markersize=3, alpha=0.3, label='Data')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Shear Stress τ_xy (Pa)')
axes[0].set_title('Coupling Mode Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3, which='both')

# Relative difference
rel_diff = 100 * np.abs(tau_xy_full - tau_xy_post) / (tau_xy_post + 1e-10)
axes[1].semilogx(t_relax, rel_diff, 'g-', linewidth=2)
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Relative Difference (%)')
axes[1].set_title('|Full - Minimal| / Minimal × 100%')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'coupling_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nMean relative difference: {np.mean(rel_diff):.2f}%")
print(f"Max relative difference: {np.max(rel_diff):.2f}%")

logger.info(f"Saved coupling comparison to {output_dir / 'coupling_comparison.png'}")

## Save Results

In [ ]:
# Save using utility function
save_fluidity_results(
    model,
    bayes_result,
    model_variant='saramito_local',
    protocol='relaxation',
    param_names=param_names
)

# Save synthetic data for reference
np.savetxt(
    output_dir / 'synthetic_relaxation_data.csv',
    np.column_stack([t_relax, tau_xy_true, tau_xy_noisy, fluidity_true]),
    header='time,stress_true,stress_noisy,fluidity',
    delimiter=',',
    comments=''
)

# Save ArviZ InferenceData
idata.to_netcdf(output_dir / 'arviz_inference.nc')

# Save summary statistics
with open(output_dir / 'mcmc_summary.txt', 'w') as f:
    f.write("# MCMC Diagnostics Summary\n\n")
    f.write(summary.to_string())
    f.write(f"\n\nMax R-hat: {rhat_max:.4f}\n")
    f.write(f"Min ESS: {ess_min:.0f}\n")

logger.info(f"\n✓ All results saved to {output_dir}")

## Key Takeaways

### Physical Insights

1. **Non-Exponential Relaxation**: The FluiditySaramitoLocal model predicts σ(t) decay that deviates from simple exponential (Maxwell) due to time-varying fluidity f(t). This creates curvature on semi-log plots — a signature of thixotropic materials.

2. **Aging Mechanism**: During quiescent relaxation (γ̇=0), fluidity evolves as df/dt = -(f-1)/t_eq, causing f → 1 (fully structured). As f decreases, the effective relaxation time λ(f) = λ₀/f increases, slowing the stress decay.

3. **Stretched Exponential**: The relaxation modulus G(t) = σ(t)/γ₀ exhibits faster decay at early times (high f, recent flow) and slower decay at late times (low f, recovered structure). This is often fit empirically as G(t) ~ exp[-(t/τ)^β] with β < 1.

4. **Tensorial Dynamics**: The UCM backbone predicts relaxation of all stress components [τ_xx, τ_yy, τ_xy]. Normal stress difference N₁ = τ_xx - τ_yy also relaxes, though it's typically small in relaxation (unlike startup flow).

5. **Von Mises Inactive**: For typical relaxation scenarios, the stress magnitude |τ| drops below τ_y quickly, so the von Mises factor α ≈ 0. Relaxation is purely viscoelastic (UCM), not plastic.

### Numerical Insights

1. **Parameter Identifiability**: Relaxation data primarily constrains:
   - **λ₀**: Initial relaxation rate (early-time decay)
   - **t_eq**: Aging timescale (curvature on semi-log plot)
   - **η₀**: Related to λ₀ via η₀ = Gλ₀ (if G known independently)

2. **Weak Constraints**: Relaxation data provides limited information on:
   - **τ_y**: Yield stress (inactive if σ < τ_y)
   - **b, n**: Rejuvenation parameters (no flow during relaxation)

3. **Coupling Modes**: For relaxation, **minimal coupling** (λ only) and **full coupling** (λ + τ_y aging) give similar results since α ≈ 0. Differences emerge in flow protocols (startup, creep).

4. **Bayesian Uncertainty**: MCMC with NLSQ warm-start achieves R-hat < 1.01 and ESS > 400 for well-constrained parameters (λ₀, t_eq). Weakly constrained parameters (b, n) show larger credible intervals.

5. **Multi-Protocol Synergy**: Combine relaxation with startup/creep/flow curve data to constrain all parameters. Relaxation alone underdetermines the full EVP model.

### Model Capabilities

1. **Tensorial Stress**: Full [τ_xx, τ_yy, τ_xy] tracking enables normal stress predictions, though N₁ is small in relaxation compared to flow.

2. **Thixotropic Signature**: Non-exponential decay G(t) distinguishes from simple Maxwell fluids, capturing microstructural recovery.

3. **JAX Efficiency**: JIT-compiled ODE integration enables fast prediction (ms per relaxation curve) for Bayesian sampling.

4. **Posterior Predictive**: Sampling from posterior enables uncertainty quantification on G(t), critical for reliability analysis.

### Experimental Connection

**Stress relaxation experiments** measure:
- Impose step strain γ₀ at t=0
- Hold strain constant: γ(t) = γ₀
- Measure stress decay σ(t)
- Compute G(t) = σ(t)/γ₀

**Common materials**:
- Laponite clay suspensions (thixotropic gels)
- Carbopol dispersions (yield-stress fluids)
- Biological gels (mucus, fibrin networks)

**Model predictions** enable:
- Quantify aging timescale t_eq from G(t) curvature
- Distinguish thixotropic (f evolves) vs viscoelastic (f constant) relaxation
- Predict long-time behavior from short-time data via calibrated f(t)
- Classify material type (gel, glass, fluid) from relaxation modality

### Next Steps

1. **Combine Protocols**: Fit startup (Tutorial 14) + relaxation simultaneously for comprehensive calibration
2. **Normal Stress Focus**: Compare N₁ relaxation with startup N₁ overshoot (both from UCM)
3. **Aging Series**: Vary waiting time before strain imposition to probe t_eq
4. **Nonlocal Extension**: Tutorial 22 for spatial gradients in f(x,t) during relaxation